# Problemas de trabajo práctico: Sesiones de doblaje
## Autor
Miguel Ángel Álvarez Cabanes
## Github
https://github.com/maalvarezcabanes/algoritmos_optimizacion

## Importación paquetes y funciones auxiliares

In [3]:
import pandas as pd
import os

## Enunciado
Se precisa coordinar el doblaje de una película. Las reglas de planificación que se deben seguir son:
1. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas.
2. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben.
3. No es posible grabar más de 6 tomas por día.
4. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible.

## Lectura de datos de tomas y actores
Los datos originales están en https://docs.google.com/spreadsheets/d/1Ipn6IrbQP4ax8zOnivdBIw2lN0JISkJG4fXndYd27U0/edit#gid=0, pero los descargo a un fichero "doblaje.xlsx" por comodidad.

A continuación leo los datos con pandas limpiando el DataFrame para solo quedarme con los datos de actores y tomas

In [17]:
df = pd.read_excel(os.path.join(".", "doblaje.xlsx"), skiprows=1).drop(["Unnamed: 11", "Total"], axis=1)
df.set_index("Toma", inplace=True)
df.drop("TOTAL", inplace=True)
df.dropna(inplace=True)
df

,1,2,3,4,5,6,7,8,9,10
Toma,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
6,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
7,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
8,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Planteamiento inicial
Para la representación de datos voy a crear una lista de días, y cada uno de los días va a estar representado por otra lista de escenas que se ruedan ese día.

La solución naïve consistiría en elegir por orden en grupos de seis tomas por día, por lo que el máximo de días debería ser cinco. Voy a crear esta solución inicial ya que puede ser usada como una referencia de "peor solución".

Nota: Voy a tratar de hacer el código usando el menor número posible de funcionalidades de librerías para tratar de que la algoritmia usada quede lo más visible posible. Por comodidad sí usaré el DataFrame para acceder a la información.

In [65]:
def coste_dia(sesiones, df, debug = False):
    coste = 0
    uso_actores = [sum(row) for index, row in df.loc[dias[0]].T.iterrows()]
    if debug:
        print(f"Sesiones: {sesiones}")
        print(f"Uso actores: {uso_actores}")
    for uso_actor in uso_actores:
        if uso_actor:
            coste += 1
    return coste

In [71]:
def coste_total(dias, df, debug = False):
    coste = 0
    for num_dia, dia in enumerate(dias):
        if debug:
            print(f"Dia: {num_dia + 1}")
        coste_aux = coste_dia(dia, df, debug)
        if debug:
            print(f"Coste día: {coste_aux}\n")
        coste += coste_aux
    return coste

In [73]:
def solucion_naive(df, debug = False):
    sesiones = list(df.index)
    dias = []
    for i in range(5):
        dias.append(sesiones[5*i: 5*(i+1)])
    
    return coste_total(dias, df, debug)

In [74]:
coste = solucion_naive(df, debug=True)
print(f"Coste total: {coste}")

Dia: 1
Sesiones: [1, 2, 3, 4, 5]
Uso actores: [2.0, 4.0, 2.0, 3.0, 3.0, 0.0, 2.0, 2.0, 0.0, 0.0]
Coste día: 7

Dia: 2
Sesiones: [6, 7, 8, 9, 10]
Uso actores: [2.0, 4.0, 2.0, 3.0, 3.0, 0.0, 2.0, 2.0, 0.0, 0.0]
Coste día: 7

Dia: 3
Sesiones: [11, 12, 13, 14, 15]
Uso actores: [2.0, 4.0, 2.0, 3.0, 3.0, 0.0, 2.0, 2.0, 0.0, 0.0]
Coste día: 7

Dia: 4
Sesiones: [16, 17, 18, 19, 20]
Uso actores: [2.0, 4.0, 2.0, 3.0, 3.0, 0.0, 2.0, 2.0, 0.0, 0.0]
Coste día: 7

Dia: 5
Sesiones: [21, 22, 23, 24, 25]
Uso actores: [2.0, 4.0, 2.0, 3.0, 3.0, 0.0, 2.0, 2.0, 0.0, 0.0]
Coste día: 7

Coste total: 35
